## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from shapely.geometry import Polygon
from pyproj import CRS
import math
from shapely.ops import cascaded_union
from shapely.geometry import Polygon

## PD Set Options

In [2]:
pd.set_option('display.max_columns',None)

## Links

DOR Use code descriptions: https://gis-mdc.opendata.arcgis.com/datasets/MDC::dor-code/explore

Palm Beach Property Appraiser Data (from the property appraiser): https://opendata2-pbcgov.opendata.arcgis.com/datasets/PBCGOV::parcels-and-property-details-webmercator/explore?location=26.643652%2C-80.455993%2C10.00&showTable=true

## Data read-in

In [3]:
# Direct from the Property Appraiser
PalmBeach_PA = pd.read_csv('Parcels_and_Property_Details_WebMercator.csv', low_memory=False)

In [4]:
# Read in geographic data
PalmBeach_geo = gpd.read_file('palmbeach_2022pin')

# Set the desired CRS
target_crs = CRS.from_epsg(4326)

# Change it to the desired crs
PalmBeach_geo = PalmBeach_geo.to_crs(target_crs)

In [6]:
# Load in Property Appraiser data
PalmBeach_data = gpd.read_file('NAL60F202201.dbf')
# Drop the empty 'geometry' column
PalmBeach_data = PalmBeach_data.drop(columns='geometry')

## Filter and Merge

In [7]:
# DOR Data
PalmBeach_merged = pd.merge(PalmBeach_geo, PalmBeach_data, left_on='PARCELNO', right_on='PARCEL_ID')
PalmBeachWellington = PalmBeach_merged[PalmBeach_merged['PHY_CITY'] == 'WELLINGTON']

In [8]:
# Property Appraiser Data filtered to a property use with 'EQUESTRIAN' in it
PalmBeach_PA_Wellington = PalmBeach_PA[PalmBeach_PA['PARCEL_NUMBER'].str.startswith('73')]
PalmBeach_PA_Wellington_Equestrian = PalmBeach_PA[PalmBeach_PA['PROPERTY_USE'].str.contains('EQUESTRIAN',na=False)]
# Get rid of unnecessary columns
PalmBeach_PA_Wellington_Equestrian = PalmBeach_PA_Wellington_Equestrian[['PARCEL_NUMBER','OWNER_NAME1','OWNER_NAME2','PADDR1','PROPERTY_USE']]

I need to merge the DOR data with the PA data, since the DOR data has previous sales information and the PA data has the equestrian property use codes

In [9]:
print(len(PalmBeachWellington))
print('--------')
print(len(PalmBeach_PA_Wellington))
print('--------')
print(len(PalmBeach_PA_Wellington_Equestrian))

22857
--------
23547
--------
1668


In [10]:
final_PalmBeach_Wellington_Equestrian = pd.merge(PalmBeachWellington, PalmBeach_PA_Wellington_Equestrian, left_on='PARCELNO', right_on='PARCEL_NUMBER')
print(len(final_PalmBeach_Wellington_Equestrian))

817


In [11]:
keep_columns = [
    'PARCELNO',
    'geometry',
    'LND_SQFOOT',
    'SALE_PRC1',
    'SALE_YR1',
    'SALE_MO1',
    'SALE_PRC2',
    'SALE_YR2',
    'SALE_MO2',
    'OWN_NAME',
    'OWN_ADDR1',
    'OWN_ADDR2',
    'OWN_CITY',
    'OWN_STATE',
    'PHY_ADDR1',
    'PHY_CITY',
    'PROPERTY_USE'
]

final_PalmBeach_Wellington_Equestrian = final_PalmBeach_Wellington_Equestrian.loc[:, keep_columns]

In [24]:
flips = final_PalmBeach_Wellington_Equestrian[final_PalmBeach_Wellington_Equestrian['SALE_YR2'] == 2022.0]
flips['pct_change'] = flips[['SALE_PRC1','SALE_PRC2']].pct_change(axis=1)['SALE_PRC2']

In [28]:
all_w_flips = PalmBeachWellington[PalmBeachWellington['SALE_YR2'] == 2022.0]
all_w_flips['pct_change'] = all_w_flips[['SALE_PRC1','SALE_PRC2']].pct_change(axis=1)['SALE_PRC2']

/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [43]:
all_pbc_flips = PalmBeach_merged[PalmBeach_merged['SALE_YR2'] == 2022.0]
all_pbc_flips['pct_change'] = all_pbc_flips[['SALE_PRC1','SALE_PRC2']].pct_change(axis=1)['SALE_PRC2']

/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [57]:
print(f"Wellington-equestrian property flips median price change: {flips['pct_change'].median():.2}")
print(f"Number of Wellington-equestrian flips: {len(flips)}")
print('-----------')
print(f"Wellington property flips (ALL) median price change: {all_w_flips['pct_change'].median():.2}")
print(f"Number of Wellington flips: {len(all_w_flips)}")

Wellington-equestrian property flips median price change: 0.01
Number of Wellington-equestrian flips: 9
-----------
Wellington property flips (ALL) median price change: 0.21
Number of Wellington flips: 272


In [31]:
all_w_flips['pct_change'].median()

0.2133641674780915

In [50]:
all_w_flips.to_csv('all_wellington_flips.csv')
flips.to_csv('wellington_equestrian_flips.csv')

In [12]:
final_PalmBeach_Wellington_Equestrian['SALE_YR2'].value_counts()

SALE_YR2
0.0       806
2022.0      9
2021.0      2
Name: count, dtype: int64

In [13]:
final_PalmBeach_Wellington_Equestrian['SALE_YR1'].value_counts()

SALE_YR1
0.0       664
2021.0     89
2022.0     64
Name: count, dtype: int64

In [19]:
# final_PalmBeach_Wellington_Equestrian.sort_values(by='SALE_PRC1',ascending=False)
final_PalmBeach_Wellington_Equestrian.groupby('SALE_YR1').agg({'SALE_PRC1':'median'})

,SALE_PRC1
SALE_YR1,
0.0,0.0
2021.0,2750000.0
2022.0,4250000.0


In [21]:
final_PalmBeach_Wellington_Equestrian.groupby('SALE_YR2').agg({'SALE_PRC2':'median'})

,SALE_PRC2
SALE_YR2,
0.0,0.0
2021.0,10.0
2022.0,4500000.0
